<a href="https://colab.research.google.com/github/Avelez164/Data-Science-Big-Data/blob/main/wrangling_tidy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CPSC 375 Classwork - Data wrangling - Joins and Tidy Data

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ProfAvery/cpsc375-classwork/blob/main/wrangling-tidy.ipynb)

- *Manager*:
- *Recorder*:
- *Presenter*:
- *Timekeeper*:

## Part 1 - Slide 10

In [1]:
import pandas as pd

band_members = pd.DataFrame(
    {"Name": ["Mick", "John", "Paul"], "Band": ["Stones", "Beatles", "Beatles"]}
)

band_instruments = pd.DataFrame(
    {"Name": ["John", "Paul", "Keith"], "Plays": ["guitar", "bass", "guitar"]}
)

<table>
<tr>
  <th>
    <code>band_members_4rows</code> (left table)
  </th>
  <th>
    <code>band_instruments_4rows</code> (right table)
  </th>
</tr>
<tr>
  <td>

| Name | Band    |
|------|---------|
| Mick | Stones  |
| John | Beatles |
| Paul | Beatles |
| Paul | Wings   |

  </td>
  <td>

| Name  | Plays  |
|-------|--------|
| John  | guitar |
| Paul  | bass   |
| Keith | guitar |
| Paul  | vocals |

  </td>
</tr>
</table>

Calculate the different joins for these two tables.

First, do them by hand

Then, confirm your results with code

* Inner

| Name | Band    | plays |
|------|---------|-------|
| John | Beatles |guitar |
| Paul | Beatles |bass   |
| Paul | Beatles |vocals |
| Paul | Wings   |bass   |
| Paul | Wings   |vocals |

* left

| Name | Band    | plays |
|------|---------|-------|
| Mick | Stones  |NaN   |
| Paul | Beatles |bass   |
| Paul | Beatles |vocals |
| Paul | Wings   |bass   |
| Paul | Wings   |vocals |


* RIght

| Name | Band    | plays |
|------|---------|-------|
| John  | guitar |
| Paul  | bass   |
| Keith | guitar |
| Paul  | vocals |



* Outer

| Name | Band    | plays |
|------|---------|-------|
| Mick | Stones  |NaN   |
| keith | Nan |bass   |
| john | Beatles |vocals |
| Paul | Beatles |bass   |
| Paul | Beatles |vocals |
| Paul | Wings   |bass   |
| Paul | Wings   |vocals |

In [7]:
band_members_4rows = pd.concat([
  band_members,
  pd.DataFrame(dict(Name=["Paul"], Band=["Wings"]))
], ignore_index=True)


band_instruments_4rows = pd.concat([
  band_instruments,
  pd.DataFrame(dict(Name=["Paul"], Plays=["vocals"]))
], ignore_index=True)
pd.merge(band_members_4rows, band_instruments_4rows, on="Name", how="inner")

,Name,Band,Plays
0,John,Beatles,guitar
1,Paul,Beatles,bass
2,Paul,Beatles,vocals
3,Paul,Wings,bass
4,Paul,Wings,vocals


In [ ]:
pd.merge(band_members_4rows, band_instruments_4rows, on="Name", how="left")

,Name,Band,Plays
0,John,Beatles,guitar
1,Keith,NaN,guitar
2,Mick,Stones,NaN
3,Paul,Beatles,bass
4,Paul,Beatles,vocals
5,Paul,Wings,bass
6,Paul,Wings,vocals


In [ ]:
pd.merge(band_members_4rows, band_instruments_4rows, on="Name", how="right")

,Name,Band,Plays
0,John,Beatles,guitar
1,Paul,Beatles,bass
2,Paul,Wings,bass
3,Keith,NaN,guitar
4,Paul,Beatles,vocals
5,Paul,Wings,vocals


In [ ]:
pd.merge(band_members_4rows, band_instruments_4rows, on="Name", how="outer", indicator = True)

,Name,Band,Plays,_merge
0,John,Beatles,guitar,both
1,Keith,NaN,guitar,right_only
2,Mick,Stones,NaN,left_only
3,Paul,Beatles,bass,both
4,Paul,Beatles,vocals,both
5,Paul,Wings,bass,both
6,Paul,Wings,vocals,both


In [ ]:
(
    pd.merge(band_members_4rows, band_instruments_4rows, how="left", indicator = True)
    .query("_merge == 'both'")
    .filter(items=["Name", "Band"])
    .drop_duplicates()
)

,Name,Band
1,John,Beatles
2,Paul,Beatles
4,Paul,Wings


## Part 2 - Slide 28

Write code to "tidy" this dataset

In [8]:
grades = pd.DataFrame(
  dict(
    CWID=(1234567, 2356902, 5005667),
    NAME=("Mary", "Lee", "Elena"),
    CPSC120=(3, 4, pd.NA),
    CPSC121=(3.3, 3, 2),
    CPSC131=(3, 4, 3.7),
    MATH338=(3, pd.NA, 3.3),
  )
)


(
    grades.melt(
        id_vars=["CWID"],
        value_vars=(["CPSC120", "CPSC121", "CPSC131", "MATH338"]),
        var_name="Course",
        value_name="Grade",
    )
    .dropna()
    #.groupby("CWID")
    #.agg({"Grade": "mean"})
    .sort_values("CWID")
    .reset_index()   #drop=True)
    .drop('index', axis=1)
 )


,CWID,Course,Grade
0,1234567,CPSC120,3
1,1234567,CPSC121,3.3
2,1234567,CPSC131,3.0
3,1234567,MATH338,3
4,2356902,CPSC120,4
5,2356902,CPSC121,3.0
6,2356902,CPSC131,4.0
7,5005667,CPSC121,2.0
8,5005667,CPSC131,3.7
9,5005667,MATH338,3.3


## Part 3 - Slide 33

`language_diversity.csv` contains data on the diversity of languages in different countries and other parameters.


In [9]:
language_diversity = pd.read_csv(
    "https://raw.githubusercontent.com/ProfAvery/cpsc375-classwork/refs/heads/main/datasets/language_diversity.csv"
)

language_diversity

,Continent,Country,Measurement,Value
0,Africa,Algeria,Langs,18.00
1,Africa,Angola,Langs,42.00
2,Oceania,Australia,Langs,234.00
3,Asia,Bangladesh,Langs,37.00
4,Africa,Benin,Langs,52.00
...,...,...,...,...
439,Asia,Vietnam,Std,1.59
440,Asia,Yemen,Std,0.00
441,Africa,Zaire,Std,1.90
442,Africa,Zambia,Std,0.67


Calculate:
 * The total number of languages in every continent

In [ ]:
(
    language_diversity
    .query('Measurement == "Langs"')
    .groupby('Continent')
    .agg({'Value': 'sum'})
)

,Value
Continent,
Africa,2513.0
Americas,877.0
Asia,1977.0
Oceania,1273.0


* Add a new variable that shows for every country its population density
  (PopulationDensity=Population/Area)

Hint: **First** convert to tidy data using `pivot()`


In [ ]:
(
    language_diversity
    .pivot(
        index=['Country', 'Continent'],
        columns='Measurement',
        values='Value'
    )
    .reset_index()
    .rename_axis(None, axis=1)
    .assign(PopulationDensity=lambda df: df.Population / df.Area)
).groupby('Continent').agg({'Langs': 'sum'})


,Langs
Continent,
Africa,2513.0
Americas,877.0
Asia,1977.0
Oceania,1273.0


Original dataset:
 * <https://github.com/jvcasillas/untidydata#language_diversity>
 * <https://www.researchgate.net/publication/222464328_Explaining_Global_Patterns_of_Language_Diversity>


## Part 4 - Slide 36

`horse_racing.csv` contains data related to horse racing licensing in New York.


In [23]:
horse_racing = pd.read_csv(
    "https://raw.githubusercontent.com/ProfAvery/cpsc375-classwork/refs/heads/main/datasets/horse_racing.csv"
)
horse_racing

,PersonID,Name,Occupation,Eligibility,Division,License,Expires
0,120384,0MAR MEHIDI,EXERCISE RIDER,ABLE TO PARTICIPATE,THOROUGHBRED,LICENSE # 1522818,4/2/2021
1,148737,AADIL S. KHAN,EXERCISE RIDER,ABLE TO PARTICIPATE,THOROUGHBRED,RECEIPT # 1462171,11/18/2019
2,200788,AALIYAH R. MCKUNE,SECURITY,ABLE TO PARTICIPATE,HARNESS,RECEIPT # 1462094,12/7/2019
3,200514,AAMER A. SALMAN,OWNER ORIGINAL,ABLE TO PARTICIPATE,THOROUGHBRED,RECEIPT # 1449814,12/31/2020
4,59203,AARON A. COTUGNO,OWNER RENEWAL,ABLE TO PARTICIPATE,THOROUGHBRED,LICENSE # 1522183,5/11/2021
...,...,...,...,...,...,...,...
24186,202965,ZOE GROSSMAN,STABLE EMPLOYEE,ABLE TO PARTICIPATE,THOROUGHBRED,RECEIPT # 1461411,9/25/2019
24187,108622,ZOILA AGUILAR,STABLE EMPLOYEE,ABLE TO PARTICIPATE,THOROUGHBRED,RECEIPT # 1461729,11/26/2019
24188,201981,ZORAIDA LEE,MUTUEL CLERK,ABLE TO PARTICIPATE,THOROUGHBRED,RECEIPT # 1456356,7/6/2019
24189,183503,ZULEIKA B. TORREALBA,OWNER RENEWAL,ABLE TO PARTICIPATE,THOROUGHBRED,RECEIPT # 1464481,12/31/2019


Transform the dataset such that:

* The License column is split into two:
  - `LicenseOrReceipt`: a categorical variable with two categories "License" and "Receipt"
  - `Number`: numeric column with the license/receipt number


In [28]:
horse_racing = pd.read_csv(
    "https://raw.githubusercontent.com/ProfAvery/cpsc375-classwork/refs/heads/main/datasets/horse_racing.csv"
)

horse_racing[['LicenseOrReciept', 'Number']] = horse_racing.License.str.split('#', expand=True)
horse_racing.LicenseOrReciept = horse_racing.LicenseOrReciept.astype('category')
horse_racing.Number = horse_racing.Number.astype('int')
horse_racing.drop('License', axis=1, inplace=True)

horse_racing[['Month', 'Day', 'Year']] = horse_racing.Expires.str.split('/', expand=True)
horse_racing.Month = horse_racing.Month.astype('int')
horse_racing.Day = horse_racing.Day.astype('int')
horse_racing.Year = horse_racing.Year.astype('int')
horse_racing.drop('Expires', axis=1, inplace=True)

horse_racing.Name = horse_racing.Name.str.replace('0', 'O')

horse_racing

,PersonID,Name,Occupation,Eligibility,Division,LicenseOrReciept,Number,Month,Day,Year
0,120384,OMAR MEHIDI,EXERCISE RIDER,ABLE TO PARTICIPATE,THOROUGHBRED,LICENSE,1522818,4,2,2021
1,148737,AADIL S. KHAN,EXERCISE RIDER,ABLE TO PARTICIPATE,THOROUGHBRED,RECEIPT,1462171,11,18,2019
2,200788,AALIYAH R. MCKUNE,SECURITY,ABLE TO PARTICIPATE,HARNESS,RECEIPT,1462094,12,7,2019
3,200514,AAMER A. SALMAN,OWNER ORIGINAL,ABLE TO PARTICIPATE,THOROUGHBRED,RECEIPT,1449814,12,31,2020
4,59203,AARON A. COTUGNO,OWNER RENEWAL,ABLE TO PARTICIPATE,THOROUGHBRED,LICENSE,1522183,5,11,2021
...,...,...,...,...,...,...,...,...,...,...
24186,202965,ZOE GROSSMAN,STABLE EMPLOYEE,ABLE TO PARTICIPATE,THOROUGHBRED,RECEIPT,1461411,9,25,2019
24187,108622,ZOILA AGUILAR,STABLE EMPLOYEE,ABLE TO PARTICIPATE,THOROUGHBRED,RECEIPT,1461729,11,26,2019
24188,201981,ZORAIDA LEE,MUTUEL CLERK,ABLE TO PARTICIPATE,THOROUGHBRED,RECEIPT,1456356,7,6,2019
24189,183503,ZULEIKA B. TORREALBA,OWNER RENEWAL,ABLE TO PARTICIPATE,THOROUGHBRED,RECEIPT,1464481,12,31,2019


* The `Expires` column is split into three:
  - `Year`, `Month`, `Day` (all integers)

In [16]:
horse_racing[['Month', 'Day', 'Year']] = horse_racing.Expires.str.split('/', expand=True)

horse_racing

,PersonID,Name,Occupation,Eligibility,Division,Expires,LicenseOrReciept,Number,Month,Day,Year
0,120384,0MAR MEHIDI,EXERCISE RIDER,ABLE TO PARTICIPATE,THOROUGHBRED,4/2/2021,LICENSE,1522818,4,2,2021
1,148737,AADIL S. KHAN,EXERCISE RIDER,ABLE TO PARTICIPATE,THOROUGHBRED,11/18/2019,RECEIPT,1462171,11,18,2019
2,200788,AALIYAH R. MCKUNE,SECURITY,ABLE TO PARTICIPATE,HARNESS,12/7/2019,RECEIPT,1462094,12,7,2019
3,200514,AAMER A. SALMAN,OWNER ORIGINAL,ABLE TO PARTICIPATE,THOROUGHBRED,12/31/2020,RECEIPT,1449814,12,31,2020
4,59203,AARON A. COTUGNO,OWNER RENEWAL,ABLE TO PARTICIPATE,THOROUGHBRED,5/11/2021,LICENSE,1522183,5,11,2021
...,...,...,...,...,...,...,...,...,...,...,...
24186,202965,ZOE GROSSMAN,STABLE EMPLOYEE,ABLE TO PARTICIPATE,THOROUGHBRED,9/25/2019,RECEIPT,1461411,9,25,2019
24187,108622,ZOILA AGUILAR,STABLE EMPLOYEE,ABLE TO PARTICIPATE,THOROUGHBRED,11/26/2019,RECEIPT,1461729,11,26,2019
24188,201981,ZORAIDA LEE,MUTUEL CLERK,ABLE TO PARTICIPATE,THOROUGHBRED,7/6/2019,RECEIPT,1456356,7,6,2019
24189,183503,ZULEIKA B. TORREALBA,OWNER RENEWAL,ABLE TO PARTICIPATE,THOROUGHBRED,12/31/2019,RECEIPT,1464481,12,31,2019


 Original dataset: <https://data.ny.gov/Government-Finance/Horse-Racing-Licensing/cz9u-yj7m/data>

## Part 5 - Slide 37

Consider the dataset: `billboard`

> Hint: convert to tidy data **first**

1. Show for each track, how many weeks it spent on the chart


In [ ]:
import statsmodels.api as sm

billboard_dataset = sm.datasets.get_rdataset("billboard", "tidyr")
billboard = billboard_dataset.data


2. List tracks in decreasing order of number of weeks spent on the chart


3. Show for each track, its top rank


4. List tracks in increasing order of its top rank


5. Show for each artist, their top rank


6. List artists in increasing order of their top rank


7. List tracks that only spent one week in the charts


8. List tracks that only spent one week in the charts along with its artist
